In [1]:
//  Copyright (c) 2020 Patrick Diehl
//
//  SPDX-License-Identifier: BSL-1.0
//  Distributed under the Boost Software License, Version 1.0. (See accompanying
//  file LICENSE_1_0.txt or copy at http://www.boost.org/LICENSE_1_0.txt)

# Exercise 1: 


In [2]:
#include <blaze/Math.h>
#include <cmath>
#include <iostream>
#include <vector>
#include<run_hpx.cpp>
#include<hpx/include/lcos.hpp>
#include<hpx/include/parallel_for_loop.hpp>

## Helpfer function for Blaze

[Blaze](https://bitbucket.org/blaze-lib/blaze/src/master/) is an open-source, high-performance C++ math library for dense and sparse arithmetic. This linrary has a HPX backend and utilzies the parallel algorithms to accelarted the linear algebara functions. Blaze is not covered in this cours and for more details we refer to

* K. Iglberger, G. Hager, J. Treibig, and U. Rüde: High Performance Smart Expression Template Math Libraries . Proceedings of the 2nd International Workshop on New Algorithms and Programming Models for the Manycore Era (APMM 2012) at HPCS 2012
* K. Iglberger, G. Hager, J. Treibig, and U. Rüde: Expression Templates Revisited: A Performance Analysis of Current Methodologies (Download). SIAM Journal on Scientific Computing, 34(2): C42--C69, 2012

In [3]:
blaze::DynamicMatrix<double> zeroMatrix(unsigned long N)
{
return blaze::DynamicMatrix<double>( N, N, 0 );
};    

In [4]:
blaze::DynamicVector<double> zeroVector(unsigned long N)
{
return blaze::DynamicVector<double>(N,0);
}; 

In [5]:
//blaze::DynamicVector<double> solve(blaze::DynamicMatrix<double> M, blaze::DynamicVector<double> )
//{


//}

## Force function

As the external load, a force function $force : \mathbb{R} \rightarrow \mathbb{R}$

 $$ force  = \begin{cases} 1, if x == 1, \\
 0 , else\end{cases}, x = [0,1]$$

In [6]:
double force(double x){
    if (x == 1)
        return 1;
    else
        return 0;
}

## Discretization

As the domain $\Omega$ we consider the intervall $[0,1]$ and discretize the interval with $n$ elements and using the spacing $h=\frac{1}{n}$ such that $x={0,1\cdot h,2\cdot h,\ldots,n-1\cdot h}$.

In [7]:
size_t n = std::pow(2,2);
double h= 1./n;
n += 1;

(unsigned long) 5


In [8]:
auto x = zeroVector(n);
for(size_t i = 0 ; i < n ; i++)
    x[i] = i * h;

In [9]:
std::cout << x ;

(           0 )
(        0.25 )
(         0.5 )
(        0.75 )
(           1 )


(std::ostream &) @0x7f737316b500


<span style="color:blue">Task 1: Replace the for loop with the hpx::for loop to fill the right-hand side $f$ in parrallel</span>.


In [10]:
auto f = zeroVector(n);

In [11]:
/*
for(size_t i = 0 ; i < n ; i++)
 {
  f[i] = force(x[i]);   
 }
*/

In [12]:
run_hpx([](){

hpx::for_loop(
	hpx::execution::par, 
	0, 
	n,
	[&](boost::uint64_t i)
		{
		   f[i] = force(x[i]); 
		}
	);
    
});

(void) @0x7f7368f5ee48


In [13]:
std::cout << f ;

(           0 )
(           0 )
(           0 )
(           0 )
(           1 )


(std::ostream &) @0x7f737316b500


<span style="color:blue">Task 2: Use aysnchronous programming to assemble the stiffness matrix using hpx::async and hpx::future</span>.

1. Finish the function assemble to fill the matrix from start to end

In [14]:
auto matrix = zeroMatrix(n);

In [15]:
/*
matrix(0,0) = 1;
for(size_t i = 1 ; i < n-1 ; i++){
    matrix(i,i-1) = -2;
    matrix(i,i) = 4;
    matrix(i,i+1) = -2;
    }
matrix(n-1,n-1) = 1;
matrix *= 1./(2*h*h);
*/

In [16]:
void assemble(blaze::DynamicMatrix<double>* matrix, size_t start, size_t end)
{
    
    for(size_t i = start ; i < end ; i++)
    {
        (*matrix)(i,i-1) = -2;
        (*matrix)(i,i) = 4;
        (*matrix)(i,i+1) = -2;
    }
}

In [17]:
run_hpx([&](){

matrix(0,0) = 1;

std::vector<hpx::future<void>> futures;

futures.push_back(hpx::async(assemble,&matrix,1,std::round(float(n)/2)));
futures.push_back(hpx::async(assemble,&matrix,std::round(float(n)/2),n-1));

hpx::wait_all(futures);

matrix(n-1,n-1) = 1;
matrix *= 1./(2*h*h);
    
});

(void) @0x7f7368f5ee48


In [18]:
std::cout << matrix ;

(            8            0            0            0            0 )
(          -16           32          -16            0            0 )
(            0          -16           32          -16            0 )
(            0            0          -16           32          -16 )
(            0            0            0            0            8 )


(std::ostream &) @0x7f737316b500
